In [7]:
import requests
import json
from datetime import datetime, timedelta

# --- Configuration ---
BASE_URL = "http://127.0.0.1:8085/api/jd" # Adjust if your app runs on a different port/host

# --- Helper Function to Print Responses ---
def print_response(response: requests.Response):
    print(f"Status Code: {response.status_code}")
    try:
        print("Response JSON:")
        print(json.dumps(response.json(), indent=2))
    except requests.exceptions.JSONDecodeError:
        print("Response Text (Not JSON):")
        print(response.text)
    print("-" * 30)

# --- Test Data ---
sample_jd_generation_payload = {
    "job_title_input": "Senior Software Engineer (Python)",
    "key_responsibilities_input": [
        "Develop and maintain scalable web applications.",
        "Collaborate with cross-functional teams.",
        "Write clean, testable code."
    ],
    "required_skills_input": ["Python", "Flask", "SQLAlchemy", "Docker"],
    "company_description_input": "A fast-growing tech company focused on AI solutions."
}

# This will be populated by the /generate endpoint or manually for testing create
generated_jd_content_sample = None

created_job_id = None # To store the ID of the created JD for subsequent tests

# --- Test Functions ---

def test_generate_jd():
    global generated_jd_content_sample
    print("Testing: POST /generate")
    try:
        response = requests.post(f"{BASE_URL}/generate", json=sample_jd_generation_payload)
        print_response(response)
        if response.status_code == 200:
            generated_jd_content_sample = response.json() # Save for create test
            print("Successfully generated JD content.")
        else:
            print("Failed to generate JD content.")
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
    print("=" * 50)

def test_create_jd():
    global created_job_id
    global generated_jd_content_sample
    print("Testing: POST / (Create JD)")

    if not generated_jd_content_sample:
        print("Skipping create test as JD generation failed or was not run.")
        # Fallback to a manual sample if generation didn't work
        generated_jd_content_sample = {
            "job_title": "Manual Test Engineer", # Gemini typically provides this as job_title
            "company_summary": "A great company.",
            "role_summary": "Testing various applications.",
            "key_responsibilities": ["Manual testing", "Bug reporting"],
            "required_qualifications": ["Attention to detail", "JIRA"],
            "preferred_qualifications": ["Automation basics"],
            "benefits": ["Health insurance", "Paid time off"]
        }
        print("Using fallback manual JD content for creation.")


    # Ensure the main job_title is present for the JDCreateRequest
    # If your Gemini output has job_title as job_title_generated, adjust here or in schema
    # My schema uses 'job_title' as the alias for 'job_title_generated'
    # The JDCreateRequest expects 'job_title' at the top level, and 'jd_content' as the object

    jd_create_payload = {
        "job_title": generated_jd_content_sample.get("job_title", "Default Test Job Title"), # Top-level job_title
        "jd_content": generated_jd_content_sample, # The whole generated structure
        "expires_at": (datetime.utcnow() + timedelta(days=30)).isoformat() + "Z" # ISO 8601 format
    }

    try:
        response = requests.post(BASE_URL, json=jd_create_payload)
        print_response(response)
        if response.status_code == 201:
            created_job_id = response.json().get("job_id")
            print(f"Successfully created JD with ID: {created_job_id}")
        else:
            print("Failed to create JD.")
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
    print("=" * 50)

def test_list_jds():
    print("Testing: GET / (List JDs)")
    try:
        response = requests.get(BASE_URL)
        print_response(response)
        if response.status_code == 200:
            print("Successfully listed JDs.")
        else:
            print("Failed to list JDs.")
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
    print("=" * 50)

def test_get_specific_jd():
    print("Testing: GET /{job_id} (Get Specific JD)")
    if not created_job_id:
        print("Skipping get specific JD test as no JD was created.")
        return

    try:
        response = requests.get(f"{BASE_URL}/{created_job_id}")
        print_response(response)
        if response.status_code == 200:
            print(f"Successfully retrieved JD ID: {created_job_id}")
        else:
            print(f"Failed to retrieve JD ID: {created_job_id}")
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
    print("=" * 50)

def test_update_jd():
    print("Testing: PUT /{job_id} (Update JD)")
    if not created_job_id:
        print("Skipping update JD test as no JD was created.")
        return

    update_payload = {
        "job_title": "Updated Senior Software Engineer (Python & AI)",
        "jd_content": { # Provide the full jd_content if you want to update it
            "job_title": "Updated Senior Software Engineer (Python & AI)", # This should also be updated
            "company_summary": "A leading AI innovation company.",
            "role_summary": "Leading AI-driven software development.",
            "key_responsibilities": [
                "Architect and develop advanced AI models.",
                "Lead a team of talented engineers.",
                "Drive innovation in machine learning applications."
            ],
            "required_qualifications": ["Python", "PyTorch", "TensorFlow", "Leadership"],
            "preferred_qualifications": ["PhD in AI/ML", "Published research"],
            "benefits": ["Competitive salary", "Stock options", "Remote work options"]
        },
        "status": "inactive",
        "expires_at": (datetime.utcnow() + timedelta(days=60)).isoformat() + "Z"
    }
    try:
        response = requests.put(f"{BASE_URL}/{created_job_id}", json=update_payload)
        print_response(response)
        if response.status_code == 200:
            print(f"Successfully updated JD ID: {created_job_id}")
        else:
            print(f"Failed to update JD ID: {created_job_id}")
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
    print("=" * 50)

def test_delete_jd():
    print("Testing: DELETE /{job_id} (Delete JD)")
    if not created_job_id:
        print("Skipping delete JD test as no JD was created.")
        return

    try:
        response = requests.delete(f"{BASE_URL}/{created_job_id}")
        print_response(response)
        if response.status_code == 200: # Your API returns 200 for successful delete
            print(f"Successfully deleted JD ID: {created_job_id}")
        elif response.status_code == 204: # Also a common success code for DELETE
            print(f"Successfully deleted JD ID: {created_job_id} (Status 204)")
        else:
            print(f"Failed to delete JD ID: {created_job_id}")
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
    print("=" * 50)

# --- Run Tests ---
if __name__ == "__main__":
    print("Starting API tests...\n")

    # Make sure your Flask app is running before executing this script.

    # 1. Test JD Generation
    test_generate_jd()

    # 2. Test JD Creation (uses content from generation if successful)
    test_create_jd()

    # 3. Test Listing JDs
    test_list_jds()

    # 4. Test Getting a Specific JD (if one was created)
    test_get_specific_jd()

    # 5. Test Updating a JD (if one was created)
    test_update_jd()

    # 6. Test Deleting a JD (if one was created)
    test_delete_jd()

    # 7. Test Get Specific JD again (should be 404 if deleted)
    if created_job_id: # Only if a JD was created and potentially deleted
        print("\nTesting: GET /{job_id} (Verify Deletion - Expect 404)")
        try:
            response = requests.get(f"{BASE_URL}/{created_job_id}")
            print_response(response)
            if response.status_code == 404:
                print(f"JD ID: {created_job_id} correctly not found after deletion.")
            else:
                print(f"JD ID: {created_job_id} was found, deletion might have failed or this is an old ID.")
        except requests.exceptions.RequestException as e:
            print(f"Request failed: {e}")
        print("=" * 50)


    print("\nAPI tests finished.")

Starting API tests...

Testing: POST /generate
Status Code: 200
Response JSON:
{
  "benefits": [
    "Competitive salary and performance bonuses.",
    "Comprehensive health, dental, and vision insurance.",
    "Generous paid time off and flexible working arrangements.",
    "Opportunities for professional development and continuous learning.",
    "Access to cutting-edge AI technologies and projects.",
    "Collaborative and inclusive company culture."
  ],
  "company_summary": "Join a fast-growing tech company at the forefront of innovation, dedicated to developing cutting-edge AI solutions that redefine industries.",
  "job_title_generated": "Senior Python Software Engineer",
  "key_responsibilities": [
    "Develop, test, and deploy highly scalable and performant web applications using Python.",
    "Collaborate closely with cross-functional teams, including product managers, data scientists, and front-end engineers, to define, design, and ship new features.",
    "Write clean, eff

C:\Users\ravibh\AppData\Local\Temp\ipykernel_12300\1397036915.py:81: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "expires_at": (datetime.utcnow() + timedelta(days=30)).isoformat() + "Z" # ISO 8601 format


Status Code: 201
Response JSON:
{
  "job_id": 5,
  "message": "JD created successfully"
}
------------------------------
Successfully created JD with ID: 5
Testing: GET / (List JDs)
Status Code: 200
Response JSON:
[
  {
    "id": 5,
    "job_title": "Default Test Job Title"
  },
  {
    "id": 1,
    "job_title": "Manual Test Engineer"
  }
]
------------------------------
Successfully listed JDs.
Testing: GET /{job_id} (Get Specific JD)
Status Code: 200
Response JSON:
{
  "created_at": "Thu, 19 Jun 2025 09:51:33 GMT",
  "expires_at": "Sat, 19 Jul 2025 09:51:33 GMT",
  "id": 5,
  "jd_content": {
    "benefits": [
      "Competitive salary and performance bonuses.",
      "Comprehensive health, dental, and vision insurance.",
      "Generous paid time off and flexible working arrangements.",
      "Opportunities for professional development and continuous learning.",
      "Access to cutting-edge AI technologies and projects.",
      "Collaborative and inclusive company culture."
    ],
 

C:\Users\ravibh\AppData\Local\Temp\ipykernel_12300\1397036915.py:148: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "expires_at": (datetime.utcnow() + timedelta(days=60)).isoformat() + "Z"


Status Code: 200
Response JSON:
{
  "created_at": "Thu, 19 Jun 2025 09:51:33 GMT",
  "expires_at": "Mon, 18 Aug 2025 09:51:39 GMT",
  "id": 5,
  "jd_content": {
    "benefits": [
      "Competitive salary",
      "Stock options",
      "Remote work options"
    ],
    "company_summary": "A leading AI innovation company.",
    "job_title_generated": "Updated Senior Software Engineer (Python & AI)",
    "key_responsibilities": [
      "Architect and develop advanced AI models.",
      "Lead a team of talented engineers.",
      "Drive innovation in machine learning applications."
    ],
    "preferred_qualifications": [
      "PhD in AI/ML",
      "Published research"
    ],
    "required_qualifications": [
      "Python",
      "PyTorch",
      "TensorFlow",
      "Leadership"
    ],
    "role_summary": "Leading AI-driven software development."
  },
  "job_title": "Updated Senior Software Engineer (Python & AI)",
  "status": "inactive"
}
------------------------------
Successfully updat